In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
from random import uniform
from random import randint
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from urllib.parse import urljoin

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time
import requests

In [2]:
from datetime import datetime

from IPython.display import clear_output

import os

from astropy.io import fits

In [3]:
directory = "/home/astrocompu21/Documentos/sim_img/download1"

In [4]:
df = pd.read_csv('gaia_100pc.csv').dropna(subset=['max', 'sum']).reset_index(drop=True)
df

,index,source_id,ra,dec,parallax,pmra,pmdec,epoch,ra_p,dec_p,cutout_status,max,sum,inside,min
0,616320,3864972938605115520,164.103190,7.002727,415.179416,-3866.338275,-2699.214988,2000.098,164.120448,7.014632,1,0.212736,1.226775,0.0,0.212736
1,263198,762815470562110464,165.830960,35.948653,392.752945,-580.057087,-4776.588719,1994.521,165.835130,35.977157,1,0.118364,0.092921,1.0,0.118364
2,685730,3796072592206250624,176.937688,0.799120,296.305308,607.299077,-1223.028355,1998.592,176.934697,0.805057,1,0.177692,0.008436,1.0,0.177692
3,380592,703790044252850688,127.450092,26.773286,279.249639,-1113.693547,-612.191073,1995.862,127.457151,26.776695,1,0.281003,0.404389,1.0,0.281003
4,813398,2552928187080872832,12.296740,5.376557,231.780000,1231.399465,-2711.882504,2009.218,12.294394,5.381661,1,0.344762,0.922290,0.0,0.344762
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76256,480261,632659544673927168,141.191779,18.727108,10.000163,24.873347,6.785537,1999.836,141.191664,18.727077,1,0.092962,-0.270498,0.0,0.092962
76257,748111,5761035592009134848,132.651076,-4.699385,10.000134,-39.836030,31.746516,2001.256,132.651237,-4.699515,1,0.240004,-0.088818,0.0,0.240004
76258,662194,3809611737872457856,162.873900,3.317327,10.000134,48.366999,3.981069,1998.516,162.873663,3.317309,1,0.274316,1.367879,1.0,0.274316
76259,137217,1407928457594520576,253.502181,47.883362,10.000129,-15.734855,49.408530,1997.258,253.502307,47.883106,1,0.285634,0.977211,1.0,0.285634


In [5]:
distances = np.array([12.5,25,50,100])
parallaxes = 1000/distances
n_stars = [len(df.query(f'parallax>{parallax}').reset_index(drop=True)) for parallax in parallaxes]

In [6]:
n_stars

[157, 1267, 10161, 76261]

In [7]:
def conv_ra(ra):
    h = int(ra/15)
    m = int((ra/15-h)*60)
    s = round(((ra/15-h)*60-m)*60,2)
    return (f'{h} {m} {s}')

def conv_dec(dec):
    sig = '+' if dec>=0 else '-'
    dec = abs(dec)
    d = int(dec)
    m = int((dec-d)*60)
    s = round(((dec-d)*60-m)*60,2)
    return (f'{sig}{d} {m} {s}')

In [8]:
def randomize_positions(ra, dec, max_shift_arcsec=60.0):
    """
    Randomiza las posiciones de las estrellas.
    
    Args:
        ra (array): Array de Ascensión Recta (RA) en grados.
        dec (array): Array de Declinación (Dec) en grados.
        max_shift_arcsec (float): Máximo desplazamiento en segundos de arco.
        n_stars (int): Número de estrellas a randomizar.

    Returns:
        ra_new, dec_new (arrays): Nuevas posiciones randomizadas (RA y Dec).
    """
    # Convertimos el máximo desplazamiento a grados
    max_shift_deg = max_shift_arcsec / 3600.0  # De segundos de arco a grados

    # Generar ángulos aleatorios (entre 0 y 360 grados) y magnitudes del desplazamiento
    angles = np.random.uniform(0, 360)  # Ángulo aleatorio en grados
    shifts = np.random.uniform(0, max_shift_deg)  # Desplazamiento aleatorio

    # Convertimos los ángulos a radianes
    angles_rad = np.deg2rad(angles)

    # Calcular desplazamientos en RA y Dec usando trigonometría
    delta_ra = shifts * np.cos(angles_rad)  # Desplazamiento en Ascensión Recta
    delta_dec = shifts * np.sin(angles_rad)  # Desplazamiento en Declinación

    # Aplicar desplazamientos a las posiciones originales
    ra_new = ra + delta_ra
    dec_new = dec + delta_dec

    return ra_new, dec_new

In [9]:
df = pd.read_csv('gaia_100pc.csv').dropna(subset=['max', 'sum']).reset_index(drop=True)
df

,index,source_id,ra,dec,parallax,pmra,pmdec,epoch,ra_p,dec_p,cutout_status,max,sum,inside,min
0,616320,3864972938605115520,164.103190,7.002727,415.179416,-3866.338275,-2699.214988,2000.098,164.120448,7.014632,1,0.212736,1.226775,0.0,0.212736
1,263198,762815470562110464,165.830960,35.948653,392.752945,-580.057087,-4776.588719,1994.521,165.835130,35.977157,1,0.118364,0.092921,1.0,0.118364
2,685730,3796072592206250624,176.937688,0.799120,296.305308,607.299077,-1223.028355,1998.592,176.934697,0.805057,1,0.177692,0.008436,1.0,0.177692
3,380592,703790044252850688,127.450092,26.773286,279.249639,-1113.693547,-612.191073,1995.862,127.457151,26.776695,1,0.281003,0.404389,1.0,0.281003
4,813398,2552928187080872832,12.296740,5.376557,231.780000,1231.399465,-2711.882504,2009.218,12.294394,5.381661,1,0.344762,0.922290,0.0,0.344762
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76256,480261,632659544673927168,141.191779,18.727108,10.000163,24.873347,6.785537,1999.836,141.191664,18.727077,1,0.092962,-0.270498,0.0,0.092962
76257,748111,5761035592009134848,132.651076,-4.699385,10.000134,-39.836030,31.746516,2001.256,132.651237,-4.699515,1,0.240004,-0.088818,0.0,0.240004
76258,662194,3809611737872457856,162.873900,3.317327,10.000134,48.366999,3.981069,1998.516,162.873663,3.317309,1,0.274316,1.367879,1.0,0.274316
76259,137217,1407928457594520576,253.502181,47.883362,10.000129,-15.734855,49.408530,1997.258,253.502307,47.883106,1,0.285634,0.977211,1.0,0.285634


In [10]:
data = df.loc[randint(0,76260)]
randomize_positions(data['ra'],data['dec'])

(177.80033915562942, 52.3371296911879)

In [11]:
chromeOptions = webdriver.ChromeOptions()
prefs = {f'download.default_directory' : f'{directory}', "safebrowsing.disable_download_protection": False}
chromeOptions.add_experimental_option("prefs", prefs)

In [12]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chromeOptions)

In [13]:
driver.get("https://third.ucllnl.org/cgi-bin/firstcutout")

In [14]:
driver.find_element(By.XPATH, "//input[@name='ImageSize']").clear()
driver.find_element(By.XPATH, "//input[@name='ImageSize']").send_keys("0.14")
driver.find_element(By.XPATH, "//input[@value='FITS File']").click()
driver.find_element(By.XPATH, "//input[@name='MaxInt']").clear()
driver.find_element(By.XPATH, "//input[@name='MaxInt']").send_keys("50")

In [15]:
def getfiles(dirpath):
    a = [s for s in os.listdir(dirpath)
         if os.path.isfile(os.path.join(dirpath, s))]
    a.sort(key=lambda s: os.path.getmtime(os.path.join(dirpath, s)))
    return a

In [16]:
positives = [1, 2, 1, 1, 2, 4, 2, 5, 4, 9, 50, 50, 43, 51]

In [17]:
positives

[1, 2, 1, 1, 2, 4, 2, 5, 4, 9, 50, 50, 43, 51]

In [18]:
k_i = 3
n_i = 1
c_i = 53350
i_i = 53368
positive_i = 204

In [ ]:
k = k_i
while k < len(n_stars):
    print("Distance:", distances[k])
    n = 0
    if k==k_i:
        n = n_i
    while n < 5:
        positive = 0
        c = 0
        i = 0
        if k==k_i and n==n_i:
            c = c_i
            i = i_i
            positive = positive_i
        print("Simulación", n+1)
        start = datetime.now()
        while c<n_stars[k]:
            data = df.loc[randint(0,76260)]
            ra, dec = randomize_positions(data['ra'],data['dec'])
            driver.find_element(By.XPATH, "//input[@name='RA']").clear()
            # introduciendo RA y DEC
            driver.find_element(By.XPATH, "//input[@name='RA']").send_keys(conv_ra(ra)+" "+conv_dec(dec))
            # submit
            driver.find_element(By.XPATH, "//input[@name='.submit']").click()
            time.sleep(0.1)
            try:
                if driver.find_element(By.XPATH, "//pre").text[:2]=="No":
                    print(i, 'not found')
                    driver.back()
            except:
                i_load = -1
                downloaded = False
                while not downloaded:
                    files = os.listdir(directory)
                    #print(files)
                    downloaded = True
                    founded = False
                    for j in range(len(files)):
                        if files[j][-4:] == "load":
                            i_load = j
                            file = files[j][:-11]
                            founded = True
                            downloaded = False
                while not founded:
                    files = os.listdir(directory)
                    for j in range(len(files)):
                        if files[j][0] == "J":
                            file = files[j]
                            founded = True
                #print(file)
                os.remove(f'{directory}/random_fits.fits')
                os.rename(directory+"/"+file, f'{directory}/random_fits.fits')
                if i%(n_stars[k]//40)==0:
                    print(i, "archivo descargado")
                    print("_____ registro actual _____")
                    print("c_i =",c)
                    print("i_i =",i)
                    print("positive_i =",positive)

                try:
                    hdul = fits.open(f'{directory}/random_fits.fits')
                    dat = hdul[0].data*1000
                    hdul.close()
                    if np.argmax(dat) in [6,7,8,11,12,13,16,17,18]:
                        max_ = np.amax(dat)
                        sum_ = np.sum(dat[1:4, 1:4])
                        if max_ > 0.5 and sum_>2.0:
                            positive += 1
                            c += 1
                            #print("positivo")
                        else:
                            c += 1
                            #print("no cumple criterios max y sum")
                    else:
                        c += 1
                        #print("no cumple criterios max inside")
                except:
                    hdul.close()
                    #print("error con fits")
            i += 1

        end = datetime.now()
        positives.append(positive)
        print(f'Resultado: {positive} positivos.')
        print(f'Tiempo de simulación: {end-start}')
        n += 1
    k += 1

Distance: 100.0
Simulación 2
53368 archivo descargado
_____ registro actual _____
c_i = 53350
i_i = 53368
positive_i = 204
55274 archivo descargado
_____ registro actual _____
c_i = 55256
i_i = 55274
positive_i = 210
56606 not found
57180 archivo descargado
_____ registro actual _____
c_i = 57161
i_i = 57180
positive_i = 221
59086 archivo descargado
_____ registro actual _____
c_i = 59067
i_i = 59086
positive_i = 227
60992 archivo descargado
_____ registro actual _____
c_i = 60973
i_i = 60992
positive_i = 237
61614 not found
62898 archivo descargado
_____ registro actual _____
c_i = 62878
i_i = 62898
positive_i = 244
64804 archivo descargado
_____ registro actual _____
c_i = 64784
i_i = 64804
positive_i = 251
66710 archivo descargado
_____ registro actual _____
c_i = 66690
i_i = 66710
positive_i = 256
68616 archivo descargado
_____ registro actual _____
c_i = 68596
i_i = 68616
positive_i = 266
70522 archivo descargado
_____ registro actual _____
c_i = 70502
i_i = 70522
positive_i = 270

In [ ]:

                files = os.listdir(directory)
                for j in range(len(files)):
                    if files[j][0] == "f":
                        os.remove(f'{directory}\\{files[j]}')

In [ ]:
print("k=",k, "n=",n,"c=",c,"i=",i,"positive=",positive)